# Undelegation Unit Test
Performing verification of the undelegation mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

stakeDelegateds: 706 events
stakeDelegatedLockeds: 474 events
stakeDelegatedWithdrawns: 331 events
allocationCloseds: 84 events
allocationCollecteds: 18 events
stakeDepositeds: 36 events
rewardsAssigneds: 84 events
delegationParametersUpdateds: 9 events
allocationCreateds: 93 events
TOTAL NUMBER OF EVENTS: 1867
You should set SIMULATION_TIME_STEPS in config.py to a minimum of 1867 to capture all events.



Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [3]:
undelegation_events_dict = undelegation_events
print(getcontext().prec)
print("EXPECTED TRUTH--Tokens locked from undelegation events:")
for timestep, undelegation_event in undelegation_events_dict.items():
    print(f"{timestep}, {undelegation_event[0]['delegator']}, {undelegation_event[0]['tokens']}")



28
EXPECTED TRUTH--Tokens locked from undelegation events:
10, 0x9327775e225a703125167eb423d246915bdc37e7, 100.000000000000000000
13, 0x57b4a5f470c85d4281f7baf5162b59243a3491df, 65257.075000000000000000
19, 0x1853bf6a892abd949aed052bb634ab96afaf31dd, 14169.714085664846000000
21, 0xf5376e847efa1ea889bfcb03706f414dade0e82c, 12488.567623993900000000
39, 0x13c0dabeb938c21524c59cdc40bcb6fdb3618754, 132706.115100000000000000
50, 0x60696f9d6ab5d99560bd04a98c718731735edd5c, 10185.289624385739620000
53, 0xcff76a3c6904456e71c40398b09bc70d3926fa6b, 895.500000000000000000
69, 0xd776a7306ee6a060cebb46b46d305e88fd39ba84, 251819.644650000000000000
70, 0x81b4687d4193916aa6cf65360bf96b1e4279278a, 6505.310000000000000000
78, 0x4861a10ad69a7a6ca29384b21ca9b0f4d875af86, 2207.484109079750000000
81, 0xe3a38bb185eebca16ac86139b4b8200c293a2dad, 5679.806919758074000000
95, 0x4e43a0cd79a9324645167e70aa6cdbdfa91a43e9, 650.029186268782600178
97, 0xfe7c442da0297313444b919ba17ca666d631d121, 18167.407271115301580346

In [4]:
print("MODELED RESULTS--Tokens locked in undelegation.")
undelegation_tokens = {}
for timestep, events in undelegation_events_dict.items():
    event = events[0]
    new_tokens = df.iloc[timestep-1].indexers[event['indexer']].delegators[event['delegator']].undelegated_tokens 
    old_tokens = df.iloc[timestep-2].indexers[event['indexer']].delegators[event['delegator']].undelegated_tokens
    undelegation_tokens[timestep] = new_tokens - old_tokens
    
    
    print(f"{timestep}, {undelegation_event[0]['delegator']}, {undelegation_tokens[timestep]}")

MODELED RESULTS--Tokens locked in undelegation.
10, 0x527b077ae93cbbd67234cd575a32c20235896d44, 99.99999999999999999999999998
13, 0x527b077ae93cbbd67234cd575a32c20235896d44, 65257.07499999999999999999999
19, 0x527b077ae93cbbd67234cd575a32c20235896d44, 14169.71408566484600000000000
21, 0x527b077ae93cbbd67234cd575a32c20235896d44, 12488.567623993900000000
39, 0x527b077ae93cbbd67234cd575a32c20235896d44, 132706.1151000000000000000000
50, 0x527b077ae93cbbd67234cd575a32c20235896d44, 10185.28962438573962000000000
53, 0x527b077ae93cbbd67234cd575a32c20235896d44, 895.4999999999999999999999998
69, 0x527b077ae93cbbd67234cd575a32c20235896d44, 251819.6446499999999999999999
70, 0x527b077ae93cbbd67234cd575a32c20235896d44, 6505.309999999999999999999999
78, 0x527b077ae93cbbd67234cd575a32c20235896d44, 2207.484109079750000000000000
81, 0x527b077ae93cbbd67234cd575a32c20235896d44, 5679.806919758073999999999999
95, 0x527b077ae93cbbd67234cd575a32c20235896d44, 650.0291862687826001784652255
97, 0x527b077ae93cbbd

In [5]:
print("UNITTEST RESULTS")
cntExact = 0
cntReallyClose = 0
cntClose = 0
cnt = 0
cntWrong = 0
for timestep, event in undelegation_events_dict.items():
    print(f"Event: Timestep={timestep}, Delegator ID={event[0]['delegator']}, Tokens Undelegated={event[0]['tokens']}")
    print(f"Model: Timestep={timestep}, Delegator ID={event[0]['delegator']}, Tokens Undelegated={undelegation_tokens[timestep]}")
    # print("Tokens Equal?", undelegation_event[0]['tokens'] == undelegation_tokens[timestep])
    # print("Tokens Within 1%?", .99 <= abs(event[0]['tokens'] / undelegation_tokens[timestep]) <= 1.01)
    try:
        ratio =  abs(event[0]['tokens'] / undelegation_tokens[timestep])
    except:
        ratio = np.inf
    exact = ratio == 1.0
    close = 0.99 <= ratio <= 1.01
    reallyClose = 0.99999 <= ratio <= 1.00001

    if exact:            
        cntExact += 1
    elif reallyClose:
        cntReallyClose += 1
    elif close:
        cntClose += 1
    else:
        cntWrong += 1
    cnt += 1
    print("Tokens Equal?", exact)
    print("Tokens Within 0.001%?", reallyClose)
    print("Tokens Within 1%?", close)
    print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
    print()
print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
    


UNITTEST RESULTS
Event: Timestep=10, Delegator ID=0x9327775e225a703125167eb423d246915bdc37e7, Tokens Undelegated=100.000000000000000000
Model: Timestep=10, Delegator ID=0x9327775e225a703125167eb423d246915bdc37e7, Tokens Undelegated=99.99999999999999999999999998
Tokens Equal? True
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 1, ReallyClose: 0, Close: 0, Wrong: 0, Total Number: 1

Event: Timestep=13, Delegator ID=0x57b4a5f470c85d4281f7baf5162b59243a3491df, Tokens Undelegated=65257.075000000000000000
Model: Timestep=13, Delegator ID=0x57b4a5f470c85d4281f7baf5162b59243a3491df, Tokens Undelegated=65257.07499999999999999999999
Tokens Equal? True
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 2, ReallyClose: 0, Close: 0, Wrong: 0, Total Number: 2

Event: Timestep=19, Delegator ID=0x1853bf6a892abd949aed052bb634ab96afaf31dd, Tokens Undelegated=14169.714085664846000000
Model: Timestep=19, Delegator ID=0x1853bf6a892abd949aed052bb634ab96afaf31dd, Tokens Undelegated=14169.7140

# Conclusion
The number of tokens locked in undelegation are close, but diverging as a percentage gradually.